# Simple, sampled  pre-processing
Only use DataframeAPI to create basic sequence of item ids and manually create a schema

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
base_path = Path("~/fall_project/MOOCCubeX/")
results_path = Path("./results")
relations_path = base_path / "relations"
#entities_path = base_path / "entities"
session2video_id_path = relations_path / "sessions_w_concepts"

In [3]:
n_users = 100000

In [4]:
%%time
sessions = pd.read_parquet(session2video_id_path)

CPU times: user 30 s, sys: 3.28 s, total: 33.3 s
Wall time: 33.6 s


In [5]:
sessions

,user_id,video_consecutive_id,session_id,seg_rep_count,video_id,local_start_time,ccid,concept_ids,concepts,fields,interaction_session
0,U_1002476,1,0,0,V_1353266,1598971366,D8EC8658CD3FC45D9C33DC5901307461,"[K_边界的受力_物理学, K_变量_物理学, K_单形_物理学, K_动量矩方程_物理学,...","[边界的受力, 变量, 单形, 动量矩方程, 观察, 轨道, 阶, 理论力学, 力的平衡，,...","[物理学, 物理学, 物理学, 物理学, 物理学, 物理学, 物理学, 物理学, 物理学, ...",U_1002476-1-0
1,U_1002476,2,0,0,V_6258948,1601736654,1D4D20B05FD0D8B99C33DC5901307461,"[K_包含已知直线_机械工程, K_辅助平面法_机械工程, K_辅助平面法求直线和平面交点_...","[包含已知直线, 辅助平面法, 辅助平面法求直线和平面交点, 辅助平面, 积聚性, 几何元素...","[机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工...",U_1002476-2-0
2,U_1002476,3,0,0,V_6258951,1601736952,292B73EF985AE70A9C33DC5901307461,"[K_z坐标轴_机械工程, K_包含已知直线_机械工程, K_侧面投影_机械工程, K_辅助...","[z坐标轴, 包含已知直线, 侧面投影, 辅助平面法求直线和平面交点, 辅助平面, 几何学原...","[机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工程, 机械工...",U_1002476-3-0
3,U_10027765,1,0,1,V_7704153,1601717800,2C72189286FF3EF99C33DC5901307461,[],[],[],U_10027765-1-0
4,U_10027765,2,0,1,V_8186213,1604113273,DC81374DA06F06D89C33DC5901307461,[],[],[],U_10027765-2-0
...,...,...,...,...,...,...,...,...,...,...,...
2009151,U_9999820,1,0,0,V_7386531,1601434095,1BAE71E966D980C29C33DC5901307461,[],[],[],U_9999820-1-0
2009152,U_9999820,2,0,0,V_7386532,1601451341,AD99106BD3AD4A789C33DC5901307461,[],[],[],U_9999820-2-0
2009153,U_9999820,3,0,0,V_7386534,1601468512,8D21D8DA741CBD639C33DC5901307461,[],[],[],U_9999820-3-0
2009154,U_9999820,4,0,0,V_7386535,1601474220,81FDDCC818CF26339C33DC5901307461,[],[],[],U_9999820-4-0


In [ ]:
rng = np.random.default_rng(53)
sampled_users = rng.choice(sessions["user_id"].unique(), n_users, replace=False)
sessions_sampled = sessions[sessions["user_id"].isin(sampled_users)]

In [ ]:
sessions_sampled

In [6]:
video_id2int = {val: i for i, val in enumerate(sessions["video_id"].unique(), start=1)}
int2video_id = {i: video_id for video_id, i in video_id2int.items()}

In [7]:
sessions["video_id"].nunique()

185637

In [8]:
MAX_VIDEO_ID = max(video_id2int.values())

In [9]:
# Basic video_id, encoding, saving 0 for padding
sessions_encoded = sessions.copy()
sessions_encoded.loc[:,"video_id"] = sessions_encoded["video_id"].map(video_id2int)

In [10]:
# Convert to list and only keep video_id for simplicity 
sessions_list = sessions_encoded.sort_values("local_start_time").groupby("user_id")["video_id"].agg(list).rename("video_id-list").reset_index()
sessions_list_hot_5 = sessions_list[sessions_list["video_id-list"].str.len() >= 5]

In [11]:
sessions_list_hot_5["video_id-list"].str.len().describe(percentiles=np.arange(.5, 1, 0.05))

count    114150.000000
mean         14.279877
std          15.113597
min           5.000000
50%           9.000000
55%          10.000000
60%          11.000000
65%          12.000000
70%          14.000000
75%          16.000000
80%          18.000000
85%          23.000000
90%          30.000000
95%          44.000000
max         570.000000
Name: video_id-list, dtype: float64

In [16]:
MAX_SEQUENCE_LENGTH = 50
pad_token = 0

In [17]:
# Slice and pad with 0
sessions_list_hot_5_sliced = sessions_list_hot_5.copy()
sessions_list_hot_5_trim = (sessions_list_hot_5_sliced["video_id-list"]
                             .str[:MAX_SEQUENCE_LENGTH]
                             .apply(lambda lst: (lst + [pad_token] * MAX_SEQUENCE_LENGTH)[:MAX_SEQUENCE_LENGTH])
                            )

In [18]:
raw_lists = np.array(sessions_list_hot_5_trim.tolist())
raw_lists

array([[ 11923,  11993,  11924, ...,      0,      0,      0],
       [ 12082,  34059,  12145, ...,      0,      0,      0],
       [ 12147,  12143,  12082, ...,      0,      0,      0],
       ...,
       [160890, 160891, 160891, ...,      0,      0,      0],
       [  2769,   2766,   2762, ...,      0,      0,      0],
       [  2769,   2766,   2763, ...,      0,      0,      0]])

In [19]:
# First encountered zero
target_idx = (raw_lists==0).argmax(1, keepdims=True) - 1 
test_values = raw_lists

In [20]:
# Fill in the value which is hidden
# ->Split into train, validation and test with last and second-to-last items

val_values = test_values.copy()
np.put_along_axis(val_values, target_idx, pad_token, axis=1)
train_values = val_values.copy()
train_mask_idx = target_idx - 1 
np.put_along_axis(train_values, train_mask_idx, pad_token, axis=1)

In [22]:
test = sessions_list_hot_5_trim.to_frame()
val = pd.Series(list(val_values), index=sessions_list_hot_5_trim.index, name=sessions_list_hot_5_trim.name).to_frame()
train = pd.Series(list(train_values), index=sessions_list_hot_5_trim.index, name=sessions_list_hot_5_trim.name).to_frame()

In [23]:
val

,video_id-list
1,"[11923, 11993, 11924, 11924, 11924, 0, 0, 0, 0..."
7,"[12082, 34059, 12145, 12031, 12036, 0, 0, 0, 0..."
10,"[12147, 12143, 12082, 12080, 34059, 12145, 121..."
18,"[2766, 2762, 2763, 2764, 2765, 0, 0, 0, 0, 0, ..."
20,"[142499, 142500, 87277, 125176, 125177, 142500..."
...,...
303624,"[2769, 2766, 2762, 2763, 2764, 0, 0, 0, 0, 0, ..."
303627,"[25718, 2221, 2222, 2223, 2953, 2953, 28783, 7..."
303628,"[160890, 160891, 160891, 160892, 160892, 16089..."
303631,"[2769, 2766, 2762, 2764, 2764, 0, 0, 0, 0, 0, ..."


In [24]:
train

,video_id-list
1,"[11923, 11993, 11924, 11924, 0, 0, 0, 0, 0, 0,..."
7,"[12082, 34059, 12145, 12031, 0, 0, 0, 0, 0, 0,..."
10,"[12147, 12143, 12082, 12080, 34059, 12145, 121..."
18,"[2766, 2762, 2763, 2764, 0, 0, 0, 0, 0, 0, 0, ..."
20,"[142499, 142500, 87277, 125176, 125177, 142500..."
...,...
303624,"[2769, 2766, 2762, 2763, 0, 0, 0, 0, 0, 0, 0, ..."
303627,"[25718, 2221, 2222, 2223, 2953, 2953, 28783, 7..."
303628,"[160890, 160891, 160891, 160892, 160892, 16089..."
303631,"[2769, 2766, 2762, 2764, 0, 0, 0, 0, 0, 0, 0, ..."


In [25]:
out_path = Path("data")

In [27]:
test.to_parquet(out_path / "test.parquet")
val.to_parquet(out_path / "val.parquet")
train.to_parquet(out_path / "train.parquet")

### Create manual schema

In [28]:
from merlin_standard_lib import Schema, Tag, ColumnSchema
from merlin.schema.tags import Tags
from merlin_standard_lib.proto.schema_bp import ValueCount, IntDomain

In [29]:
schema = Schema([
        ColumnSchema.create_categorical("video_id-list", MAX_VIDEO_ID, 
                                        value_count= ValueCount(0, MAX_SEQUENCE_LENGTH),
                                        min_index=0,
                                        tags=[Tag.CATEGORICAL, Tag.LIST, Tag.ITEM, Tag.ITEM_ID]
                                       )
    ])

In [30]:
with open("mooc_schema.pb", "w") as file:
    file.write(schema.to_proto_text())

## Test run with Transformers4Rec

In [31]:
import os

#os.environ["CUDA_VISIBLE_DEVICES"]="0"
import glob
import torch 

from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory
from transformers4rec.config.transformer import BertConfig

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
schema

[{'name': 'video_id-list', 'value_count': {'max': '50'}, 'type': 'INT', 'int_domain': {'name': 'video_id-list', 'max': '185637', 'is_categorical': True}, 'annotation': {'tag': ['list', 'categorical', 'item', 'item_id']}}]

In [37]:
#schema = Schema().from_proto_text("sample_schema.pb")
#schema

In [33]:
# Define input module - No continous features -> No need for projection 
input_module = tr.TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=MAX_SEQUENCE_LENGTH,
        #continuous_projection=64,
        masking="mlm",
        #d_output=100,
)

In [42]:
# Configuration adapted to what was used in Fall-report
transformer_config = BertConfig.build(
    d_model=64, n_head=4, n_layer=3, total_seq_length=MAX_SEQUENCE_LENGTH,
    initializer_range=0.02, hidden_dropout_prob=0.2, intermediate_size=128, attention_probs_dropout_prob = 0.2,
    layer_norm_eps=1e-12
)

# Define the evaluation top-N metrics and the cut-offs
metrics = [NDCGAt(top_ks=[5, 10], labels_onehot=True),  
           RecallAt(top_ks=[5, 10], labels_onehot=True),
           AvgPrecisionAt(top_ks=[5, 10], labels_onehot=True)
          ]
prediction_task = tr.NextItemPredictionTask(
    weight_tying=True, metrics=metrics, loss=torch.nn.CrossEntropyLoss()
)
model = transformer_config.to_torch_model(input_module, prediction_task)
model

Model(
  (heads): ModuleList(
    (0): Head(
      (body): SequentialBlock(
        (0): TabularSequenceFeatures(
          (to_merge): ModuleDict(
            (categorical_module): SequenceEmbeddingFeatures(
              (filter_features): FilterFeatures()
              (embedding_tables): ModuleDict(
                (video_id-list): Embedding(185638, 64, padding_idx=0)
              )
            )
          )
          (_aggregation): ConcatFeatures()
          (_masking): MaskedLanguageModeling()
        )
        (1): TansformerBlock(
          (transformer): BertModel(
            (embeddings): BertEmbeddings(
              (word_embeddings): Embedding(1, 64, padding_idx=0)
              (position_embeddings): Embedding(52, 64)
              (token_type_embeddings): Embedding(2, 64)
              (LayerNorm): LayerNorm((64,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (encoder): BertEncoder(
              

In [35]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer

In [ ]:
import random as pyrandom

In [52]:
#Setting all seeds, maybe redundant
seed = 53
torch.manual_seed(seed)
np.random.default_rng(seed)
pyrandom.seed(seed)

# Set hyperparameters for training 
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64
train_args = T4RecTrainingArguments(data_loader_engine='merlin', 
                                    #dataloader_drop_last = True,
                                    #gradient_accumulation_steps = 1,
                                    per_device_train_batch_size = TRAIN_BATCH_SIZE,#per_device_train_batch_size, 
                                    per_device_eval_batch_size = EVAL_BATCH_SIZE,
                                    output_dir = "./bert4rec-exp-repro", 
                                    save_total_limit=5,
                                    logging_steps=500,
                                    report_to = [],
                                    num_train_epochs=27,
                                    max_sequence_length=MAX_SEQUENCE_LENGTH, 
                                    no_cuda=False,
                                    seed=seed,
                                    save_strategy="steps",
                                    evaluation_strategy="steps",
                                    eval_steps=500,
                                    save_steps=500,
                                    load_best_model_at_end=True,
                                    metric_for_best_model="/next-item/ndcg_at_10",
                                    # Optimizer - Adapted from Keras Adam default params
                                    max_grad_norm=None,
                                    learning_rate=0.001,
                                    #lr_scheduler_type='cosine', 
                                    #learning_rate_num_cosine_cycles_by_epoch=1.5,
                                   )

trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

PyTorch: setting up devices


In [54]:
trainer.train_dataset_or_path = str(out_path / "train.parquet")
trainer.eval_dataset_or_path = str(out_path / "val.parquet")

In [56]:
# INITIAL EVAL
trainer.evaluate()

{'eval_/next-item/ndcg_at_5': 0.08258748799562454,
 'eval_/next-item/ndcg_at_10': 0.09411000460386276,
 'eval_/next-item/recall_at_5': 0.11929041147232056,
 'eval_/next-item/recall_at_10': 0.15493649244308472,
 'eval_/next-item/avg_precision_at_5': 0.07049890607595444,
 'eval_/next-item/avg_precision_at_10': 0.07525069266557693,
 'eval_/loss': 9.08997917175293,
 'eval_runtime': 26.4106,
 'eval_samples_per_second': 4323.12,
 'eval_steps_per_second': 67.549}

In [ ]:
#trainer.reset_lr_scheduler()
trainer.train()

***** Running training *****
  Num examples = 114176
  Num Epochs = 27
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 48168


Step,Training Loss,Validation Loss,/next-item/ndcg At 5,/next-item/ndcg At 10,/next-item/recall At 5,/next-item/recall At 10,/next-item/avg Precision At 5,/next-item/avg Precision At 10,/loss
500,7.773600,No log,0.091805,0.105463,0.132939,0.174998,0.078308,0.083976,8.833998
1000,7.416900,No log,0.114734,0.130956,0.165256,0.215208,0.098094,0.104829,8.499044
1500,7.090700,No log,0.135717,0.150081,0.190153,0.234446,0.117661,0.123612,8.342600
2000,6.671700,No log,0.143725,0.160443,0.207762,0.259422,0.122582,0.129490,8.176430
2500,6.298300,No log,0.158241,0.178126,0.231056,0.292002,0.134285,0.142596,7.919992
3000,6.081300,No log,0.184462,0.205021,0.255217,0.318520,0.161142,0.169677,7.723702
3500,5.859100,No log,0.192636,0.215473,0.268647,0.338887,0.167608,0.177105,7.587871
4000,5.503800,No log,0.200337,0.224054,0.286640,0.359571,0.171881,0.181748,7.569904
4500,5.336800,No log,0.222527,0.246318,0.307306,0.380482,0.194614,0.204506,7.423180
5000,5.243400,No log,0.232484,0.258220,0.319501,0.398668,0.203886,0.214586,7.292773


Saving model checkpoint to ./bert4rec-exp-repro/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./bert4rec-exp-repro/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./bert4rec-exp-repro/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./bert4rec-exp-repro/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./bert4rec-exp-repro/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to ./bert4rec-exp-repro/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [bert4rec-exp-repro/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./bert4rec-exp-repro/checkpoint-3500
Trainer.model is not a `PreTrainedModel`, 

In [ ]:
# Testing Training performance
trainer.eval_dataset_or_path = str(out_path / "test.parquet")
trainer.evaluate()

In [111]:
trainer.eval_dataset_or_path = str(out_path / "test.parquet")
trainer.evaluate()

{'eval_/next-item/ndcg_at_5': 0.29387301206588745,
 'eval_/next-item/ndcg_at_10': 0.31657740473747253,
 'eval_/next-item/recall_at_5': 0.3760093152523041,
 'eval_/next-item/recall_at_10': 0.4459269642829895,
 'eval_/next-item/avg_precision_at_5': 0.26673415303230286,
 'eval_/next-item/avg_precision_at_10': 0.2761603891849518,
 'eval_/loss': 9.373533248901367,
 'eval_runtime': 7.9329,
 'eval_samples_per_second': 4767.966,
 'eval_steps_per_second': 74.499}